# Multi Criteria Software Component Asssigment Problem Solver

The core challenge lies in minimizing conflicting objectives (e.g., communication cost vs. hardware cost) while ensuring strict safety and resource constraints are met.

## Decision we need to take:
1. Is it design time thing or should we also include runtime organization of SC? 
2. What should be the design parameters?
3. Do we need specific use case? 
4. How to test it? and where to test it? 
5. Do we need to use specific tools? 
6. Do we need to fous on specific aspect like *Latency*? 
7. tink about vendor compatibility?


## Potential ways to improve and have academic work:
1. **Predictive optimization with ML to predict input data**: By using ML techniques on previous data to predict requriement of *CPU*/*GPU* etc. of SC.
2. **"Warm Start" with Graph Neural Networks (GNN)**: if NUM_COMPONENTS = 40 there is no problem for PulP or Z3 to solve it. But once number of *SC* and *ECU* goes higher its become a problem...  
3. **Dynamic Resource allocation durign Runtime**: We can not use optimization tool during the Runtime because it is slow  and what happen if novel situation occurs (OTA). We could utilize ML/RL methods since one they trained can give results in ms. In safety-critical systems, dynamic allocation could creates non-deterministic behavior. Instead of calculating a new configuration on the fly, use Design Time Optimization to pre-calculate "Fallback Scenarios".

4. **Include uncertainities**: 
   * **Aleatoric Uncertainty:** The software's runtime behavior. An image processing algorithm might use 90% of the CPU on the highway, but 10% when parked. This is a range, not a fixed number.
   * **Epistemic Uncertainty:**  The code isn't finished yet. You modeled it as 50 MB RAM consumption. This is a risk of estimation error.
   * **Evolutionary Uncertainty:** What will happen if an OTA (Over-the-Air) update comes?
   * **To model uncertainities:** We can use *Soft Constraints* and *Slack Variables*. We could utilize ***Lexicographic*** optimization 
        - Safety First: Isolation ve ASIL constraints.
        - Robustness Second: Minimize Slack variables.
        - Cost Third: Lower the cost.

## Current Takeaways
* Quality of the optimization depends on quality of guess about **REQ_CPU**, ***COMM_VOLUME***, etc.
* We could create gaussian distribution about **REQ_CPU**, **COMM_VOLUME** instead of deterministic number.
* When NUM_COMPONENTS = 40 it can solve the problem but it can not solve the problem NUM_COMPONENTS = 400
* We can have *WARM_START* with maybe GNN or other approach to have starting point for ILP
* We should normalize the values since they are in different units, that harms the gradients... 

## Dummy Data Generation

### Hardware Features:
1) **HW_HSM (Hardware Security Module)**: a dedicated physical device that securely stores, manages, and uses cryptographic keys
2) **HW_FLEX (FlexRay Controller)**: a hardware component that executes the FlexRay communication protocol, handling all tasks related to data transmission
3) **HW_CANFD (CAN)** : Classic CAN
4) **HW_ETH (Automotive Ethernet)** : Classic Ethernet

### Units:
1) **CPU**: Unit is Instructions Per Second
2) **GPU**: Unit is FLOPS
3) **RAM**: Unit is MiB
4) **ROM**: Unit is MiB


### ECU Details:
We have ECU tiers: **HPC**, **Gateway**, **Safety** those are not directly taking care of by the optimization but those are kinda template to determine features of the ECUs.

### Component Types:
We have component types, thoose are also not taking into account again those are just template 
1) **PERCEPTION**: AI etc, High computation power, High Bandwidth, Requires a **GPU**, **HW_ETH** would require, ASIL b/w **QM** and **B**.
2) **CONTROL**: Chassis, Brake, Motor, Hard Real-Time (Deterministic), **HW_FLEX** / **HW_HSM**, low CPU requirement, ASIL D.
3) **BODY**: Klima, Doors, Lights., Low Resources, ASIL QM/A/B.
4) **INFOTAINMENT**: Screen, maps, etc.





In [ ]:
import pulp as lp
import numpy as np
import random
from typing import List, Dict, Tuple

# --- Configuration for Generation ---
NUM_COMPONENTS = 40  
NUM_ECUS = 10          
RANDOM_SEED = 42 

# Definitions 
COMPONENTS = [f'C_{i}' for i in range(1, NUM_COMPONENTS + 1)]
ECUS = [f'E_{j}' for j in range(1, NUM_ECUS + 1)]
M = len(COMPONENTS)

# --- Percentages for Generation ---
# Supply (ECUs) - Counts
COUNT_HPC = 3
COUNT_SAFETY = 4
COUNT_GATEWAY = 3

# Demand (Components) - Weights
WEIGHT_PERCEPTION = 0.25
WEIGHT_CONTROL = 0.25
WEIGHT_BODY = 0.30
WEIGHT_INFOTAINMENT = 0.2  

# ASIL Mapping 
ASIL_MAP = {'QM': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4}

def generate_robust_data(cnt_hpc, cnt_safety, cnt_gateway, w_perception, w_control, w_body, w_infotainment):
    """Generates synthetic data"""
    
    if RANDOM_SEED is not None:
        random.seed(RANDOM_SEED)
    
    # --- 1. Define ECU Profiles (The Supply) ---
    # Create list based on counts
    ecu_types_list = ['HPC']*cnt_hpc + ['SAFETY']*cnt_safety + ['GATEWAY']*cnt_gateway
    
    while len(ecu_types_list) < NUM_ECUS:
        ecu_types_list.append(random.choice(['HPC', 'SAFETY', 'GATEWAY']))
    
    random.shuffle(ecu_types_list) 
    
    # Initialize Dictionaries
    max_cpu, max_ram, max_rom, max_gpu = {}, {}, {}, {}
    ecu_asil, ecu_caps, ecu_tiers =  {}, {}, {}
    base_cost = {}

    for idx, tier in enumerate(ecu_types_list):
        j = ECUS[idx] 
        ecu_tiers[j] = tier
        
        if tier == 'HPC':
            max_cpu[j] = 250000.0  # 250k DMIPS
            max_gpu[j] = 500.0     # 500 TOPS (AI)
            max_ram[j] = 65536.0   # 64 GB RAM
            max_rom[j] = 256000.0  # 256 GB Storage
            base_cost[j] = 200.0
            ecu_asil[j] = 'B' 
            ecu_caps[j] = ['HW_ETH']
            
        elif tier == 'SAFETY':
            max_cpu[j] = 16000.0   # 16k DMIPS
            max_gpu[j] = 0.0       # No AI Accel
            max_ram[j] = 24.0      # 24 MB SRAM
            max_rom[j] = 64.0      # 64 MB Flash
            base_cost[j] = 80.0
            ecu_asil[j] = 'D' 
            ecu_caps[j] = ['HW_HSM', 'HW_FLEX', 'HW_CANFD']
            
        else: # GATEWAY
            max_cpu[j] = 8000.0    # 8k DMIPS
            max_gpu[j] = 0.0
            max_ram[j] = 8.0       # 8 MB SRAM
            max_rom[j] = 16.0      # 16 MB Flash
            base_cost[j] = 30.0
            ecu_asil[j] = 'B'
            ecu_caps[j] = ['HW_HSM', 'HW_CANFD','HW_ETH']

    # --- 2. Define Component Profiles (The Demand) ---
    req_cpu, req_ram, req_rom, req_gpu = {}, {}, {}, {}
    req_asil, req_hw = {}, {}
    comp_types = {} 
    
    for i in COMPONENTS:
        # Use the weights passed as arguments
        ctype = random.choices(
            ['PERCEPTION', 'CONTROL', 'BODY', "INFOTAINMENT"], 
            weights=[w_perception, w_control, w_body, w_infotainment] 
        )[0]
        comp_types[i] = ctype
        
        if ctype == 'PERCEPTION':
            # High Load AI Task
            req_cpu[i] = random.uniform(15000, 40000) # 15k-40k DMIPS
            req_gpu[i] = random.uniform(20, 150)       # 20-80 TOPS
            req_ram[i] = random.uniform(2048*2, 8192*2)   # 2-8 GB RAM
            req_rom[i] = random.uniform(1000, 5000)   # 1-5 GB Storage
            req_asil[i] = random.choice(['QM', 'B'])
            req_hw[i] = ['HW_ETH']
            
        elif ctype == 'CONTROL':
            # Critical Real-Time Task
            req_cpu[i] = random.uniform(500, 3000)    # 500-3k DMIPS
            req_gpu[i] = 0.0
            req_ram[i] = random.uniform(1, 5)         # 1-5 MB
            req_rom[i] = random.uniform(2, 10)        # 2-10 MB
            req_asil[i] = 'D' 
            req_hw[i] = ['HW_FLEX'] 

        elif ctype == 'INFOTAINMENT':
            # Medium Load Task
            req_cpu[i] = random.uniform(5000, 20000)  # 2k-10k DMIPS
            req_gpu[i] = random.uniform(10, 80)        # 1-10 TOPS
            req_ram[i] = random.uniform(2048*2, 8192*2)    # 512 MB - 2 GB
            req_rom[i] = random.uniform(1000, 2000)    # 500 MB - 2 GB
            req_asil[i] = random.choice(['QM', 'A'])
            req_hw[i] = ['HW_ETH']
            
        else: # BODY
            # Simple Logic
            req_cpu[i] = random.uniform(50, 300)      # 50-300 DMIPS
            req_gpu[i] = 0.0
            req_ram[i] = random.uniform(0.1, 1.0)     # <1 MB
            req_rom[i] = random.uniform(0.5, 2.0)     # <2 MB
            req_asil[i] = random.choice(['QM', 'A'])
            req_hw[i] = [] 

    # --- 3. Comm Matrix ---
    comm_vol = {}
    for i in COMPONENTS:
        for k in COMPONENTS:
            if i < k and random.random() < 0.3: # 10% chance of communication 
                comm_vol[(i, k)] = random.randint(5, 50)
                comm_vol[(k, i)] = comm_vol[(i, k)]

    # --- 4. Critical Pairs ---
    critical_pairs = []
    safety_comps = [c for c, a in req_asil.items() if a == 'D']
    if len(safety_comps) >= 2:
        c1, c2 = random.sample(safety_comps, 2)
        critical_pairs.append((c1, c2))

    return (max_cpu, max_ram, max_rom, max_gpu, base_cost, ecu_caps, ecu_asil, ecu_tiers,
            req_cpu, req_ram, req_rom, req_gpu, req_asil, req_hw, comp_types,
            comm_vol, critical_pairs)

In [2]:
# --- EXECUTE GENERATION ---
# 1. Call the function with the correct parameters
# 2. Unpack ALL 17 return values including the new ROM/GPU data
(MAX_CPU, MAX_RAM, MAX_ROM, MAX_GPU, ECU_BASE_COST, ECU_HW_CAPABILITIES, ECU_ASIL_CAPABILITY, ECU_TIERS,
 REQ_CPU, REQ_RAM, REQ_ROM, REQ_GPU, REQ_ASIL, REQ_HW, COMP_TYPES,
 COMM_VOLUME, CRITICAL_PAIRS_TO_SEPARATE) = generate_robust_data(
    COUNT_HPC, COUNT_SAFETY, COUNT_GATEWAY, 
    WEIGHT_PERCEPTION, WEIGHT_CONTROL, WEIGHT_BODY, WEIGHT_INFOTAINMENT
)

# --- PRE-SOLVE DIAGNOSTICS ---
print(f"\n{'='*60}")
print(f"      SCENARIO SUMMARY (Seed: {RANDOM_SEED})")
print(f"{'='*60}")

# Check 1: CPU Supply vs Demand (DMIPS)
total_cpu_supply = sum(MAX_CPU.values())
total_cpu_demand = sum(REQ_CPU.values())
print(f"CPU Supply (DMIPS): {total_cpu_supply:,.0f} | Demand: {total_cpu_demand:,.0f}")
if total_cpu_demand > total_cpu_supply:
    print("FAILURE: Global CPU Shortage")

# Check 2: GPU Supply vs Demand (TOPS)
# Now we compare actual TOPS, not just CPU power of GPU nodes.
total_gpu_supply = sum(MAX_GPU.values())
total_gpu_demand = sum(REQ_GPU.values())
print(f"GPU Supply (TOPS):  {total_gpu_supply:,.0f} | Demand: {total_gpu_demand:,.0f}")

if total_gpu_demand > total_gpu_supply:
    print("FAILURE: Not enough AI Acceleration (TOPS)!")
elif total_gpu_demand > total_gpu_supply * 0.9:
    print("WARNING: GPU Saturation > 90%")
else:
    print("GPU Supply Healthy")

# Check 3: ASIL D Constraints
safety_ecus = [e for e, lvl in ECU_ASIL_CAPABILITY.items() if lvl == 'D']
safety_comps = [c for c, lvl in REQ_ASIL.items() if lvl == 'D']
saf_supply = sum(MAX_CPU[e] for e in safety_ecus)
saf_demand = sum(REQ_CPU[c] for c in safety_comps)

print(f"Safety Nodes: {len(safety_ecus)} ECUs | Critical Tasks: {len(safety_comps)}")
if saf_demand > saf_supply:
     print("FAILURE: Safety Tier CPU Overload")
else:
     print("Safety Tier Supply Healthy")

# --- DETAILED DATA INSPECTOR ---
print(f"\n{'='*80}")
print(f"      DETAILED ECU DATA (Supply)")
print(f"{'='*80}")
# Adjusted formatting for larger numbers
print(f"{'ECU':<6} | {'TIER':<8} | {'ASIL':<5} | {'CPU(DMIPS)':<11} | {'GPU(TOPS)':<10} | {'RAM(MB)':<9} | {'ROM(MB)':<9} | {'COST':<5}")
print("-" * 80)
for j in ECUS:
    print(f"{j:<6} | {ECU_TIERS[j]:<8} | {ECU_ASIL_CAPABILITY[j]:<5} | {MAX_CPU[j]:<11.0f} | {MAX_GPU[j]:<10.0f} | {MAX_RAM[j]:<9.0f} | {MAX_ROM[j]:<9.0f} | {ECU_BASE_COST[j]:<5.0f}")

print(f"\n{'='*80}")
print(f"      DETAILED COMPONENT DATA (Demand)")
print(f"{'='*80}")
print(f"{'COMP':<6} | {'TYPE':<12} | {'ASIL':<5} | {'CPU':<7} | {'GPU':<5} | {'RAM':<6} | {'ROM':<6} | {'REQ HW'}")
print("-" * 80)
# Showing first 15 components to keep log readable, remove slice [:15] to see all
for i in COMPONENTS: 
    hw = ",".join(REQ_HW[i]) if REQ_HW[i] else "-"
    # Format floats to 1 decimal place
    print(f"{i:<6} | {COMP_TYPES[i]:<12} | {REQ_ASIL[i]:<5} | {REQ_CPU[i]:<7.0f} | {REQ_GPU[i]:<5.1f} | {REQ_RAM[i]:<6.1f} | {REQ_ROM[i]:<6.1f} | {hw}")

print("\nData generation complete.")


      SCENARIO SUMMARY (Seed: 42)
CPU Supply (DMIPS): 838,000 | Demand: 350,190
GPU Supply (TOPS):  1,500 | Demand: 1,028
GPU Supply Healthy
Safety Nodes: 4 ECUs | Critical Tasks: 11
Safety Tier Supply Healthy

      DETAILED ECU DATA (Supply)
ECU    | TIER     | ASIL  | CPU(DMIPS)  | GPU(TOPS)  | RAM(MB)   | ROM(MB)   | COST 
--------------------------------------------------------------------------------
E_1    | GATEWAY  | B     | 8000        | 0          | 8         | 16        | 30   
E_2    | SAFETY   | D     | 16000       | 0          | 24        | 64        | 80   
E_3    | HPC      | B     | 250000      | 500        | 65536     | 256000    | 200  
E_4    | GATEWAY  | B     | 8000        | 0          | 8         | 16        | 30   
E_5    | SAFETY   | D     | 16000       | 0          | 24        | 64        | 80   
E_6    | SAFETY   | D     | 16000       | 0          | 24        | 64        | 80   
E_7    | GATEWAY  | B     | 8000        | 0          | 8         | 16        | 

1. **Decision Variables**: 
    We define the primary binary assigment variable for Software Component (SC) and a secondary binary variable for ECU utilization.

    $x_{ij}$: Binary variable, $x_{ij}$ = 1 if **SC** $i$ is assigned to **ECU** $j$, and 0 otherwise.

    $y_j$: Binary variable, $y_j$ = 1 if **ECU** $j$ is utilized (at least one component asssigned to it) otherwise is 0. 

In [3]:
# Initialize the Model
model = lp.LpProblem("ECU_Assignment_Optimization", lp.LpMinimize)

# 1. Assignment Variable (x_ij): x[i][j] = 1 if Component i is assigned to ECU j
x = lp.LpVariable.dicts("Assign", (COMPONENTS, ECUS), 0, 1, lp.LpBinary)

# 2. ECU Usage Variable (y_j): y[j] = 1 if ECU j is utilized
y = lp.LpVariable.dicts("ECU_Used", ECUS, 0, 1, lp.LpBinary)

Convert Non-Linear Optimization to linear one through linearization

(Glover Linearization): Instead of multiplication of two different variables, we define ($w$). $w = x_{ij} \cdot x_{kl}$.
To satisfy this we add following constraints:
1) $w \le x_{ij}$ (if $x_{ij}$ 0 , $w$ = 0.)
2) $w \le x_{kl}$ (if $x_{kl}$ 0 , $w$ = 0.)
3) $w \ge x_{ij} + x_{kl} - 1$ (if both 1: $1+1-1 = 1 \Rightarrow w \ge 1$. Since it is binary $w=1$.)

In [4]:
# 1.2. Linearization Variables for Communication Cost (Z1)

# w_ijkl: Binary variable, w[i][k][j][l] = 1 if C_i is on E_j AND C_k is on E_l
w = lp.LpVariable.dicts("Comm_Link", 
                        ((i, k, j, l) for i, k in COMM_VOLUME.keys() for j in ECUS for l in ECUS), 
                        0, 1, lp.LpBinary)

# --- Linearization Constraints for w_ijkl ---
# We iterate over the same keys used to define the variable w
for i, k in COMM_VOLUME.keys():
    for j in ECUS:
        for l in ECUS:
            if j != l: # Only interested in inter-ECU communication
         
                w_key = (i, k, j, l)
                
                # 1. w <= x_ij
                model += w[w_key] <= x[i][j], f"L1_{i}_{k}_{j}_{l}"
                
                # 2. w <= x_kl
                model += w[w_key] <= x[k][l], f"L2_{i}_{k}_{j}_{l}"
                
                # 3. w >= x_ij + x_kl - 1
                model += w[w_key] >= x[i][j] + x[k][l] - 1, f"L3_{i}_{k}_{j}_{l}"

## Constraints (Feasibility/Design Criteria): 
These are the strict rules that must be satisfied for any valid solution.


### C1: Unique Assignment: 
Each component _i_ must be assigned to exactly one **ECU** 

$$\sum_{j=1}^{J} x_{ij} = 1 \quad \forall i \in \{1, \dots, I\}$$

In [5]:
for i in COMPONENTS:
    model += lp.lpSum(x[i][j] for j in ECUS) == 1, f"C1_Unique_Assignment_{i}"

### C2: Resource Capacity: 
Total required **_CPU/RAM/ROM_** resources by assigned components must not exceed the ECU's capacity.
$$\sum_{i=1}^{I} (\text{ReqResource}_{i, r} \cdot x_{ij}) \le \text{MaxCapacity}_{j, r} \quad \forall j \in \{1, \dots, J\}, \forall r \in \{CPU,RAM,ROM\}$$

In [6]:
for j in ECUS:
    # CPU Constraint
    cpu_load = lp.lpSum(REQ_CPU[i] * x[i][j] for i in COMPONENTS)
    model += cpu_load <= MAX_CPU[j], f"C2_CPU_{j}"
    
    # RAM Constraint
    ram_load = lp.lpSum(REQ_RAM[i] * x[i][j] for i in COMPONENTS)
    model += ram_load <= MAX_RAM[j], f"C2_RAM_{j}"
    
    # ROM Constraint
    rom_load = lp.lpSum(REQ_ROM[i] * x[i][j] for i in COMPONENTS)
    model += rom_load <= MAX_ROM[j], f"C2_ROM_{j}"

    # GPU Constraint
    gpu_load = lp.lpSum(REQ_GPU[i] * x[i][j] for i in COMPONENTS)
    model += gpu_load <= MAX_GPU[j], f"C2_GPU_{j}"

### C3: Safety Isolation (SPoF):

Critical redundant components ($i, k$) cannot be placed on the same ECU to prevent a Single Point of Failure.
$$x_{ij} + x_{kj} \le 1 \quad \forall j \in \{1, \dots, J\} \text{ if } (i, k) \text{ is a critical redundant pair}$$

In [7]:
for c1, c2 in CRITICAL_PAIRS_TO_SEPARATE:
    for j in ECUS:
        # If C1 is assigned to E_j (1), C2 must not be assigned to E_j (0).
        model += x[c1][j] + x[c2][j] <= 1, f"C3_SPoF_Separation_{c1}_{c2}_{j}"

### C4: Hardware Compatibility:
ConstraintComponent $i$ can only be assigned to ECU $j$ if the specific hardware or peripheral requirements (${Req_{HW}}^i$) of $i$ are available ($\text{AvailHW}_j$) on $j$.$$x_{ij} = 0 \quad \text{if } \text{ReqHW}_i \nsubseteq \text{AvailHW}_j$$

In [8]:
# --- C4: Hardware Compatibility Constraints ---
# Logic: If Required_HW(i) is NOT a subset of Available_HW(j), then x[i][j] must be 0.

for i in COMPONENTS:
    required_features = set(REQ_HW.get(i, []))
    print(f"Component {i} requires features: {required_features}")

    for j in ECUS:

        available_features = set(ECU_HW_CAPABILITIES.get(j, []))
        if i == 'C_1':  # Example condition to limit output
            print(f"ECU {j} has capabilities: {available_features}")
        
        # Check if requirements are a subset of capabilities
        if not required_features.issubset(available_features):
            # Constraint: Force assignment to 0 (Impossible assignment)
            model += x[i][j] == 0, f"C4_HW_Incompatible_{i}_{j}"

Component C_1 requires features: set()
ECU E_1 has capabilities: {'HW_CANFD', 'HW_ETH', 'HW_HSM'}
ECU E_2 has capabilities: {'HW_CANFD', 'HW_HSM', 'HW_FLEX'}
ECU E_3 has capabilities: {'HW_ETH'}
ECU E_4 has capabilities: {'HW_CANFD', 'HW_ETH', 'HW_HSM'}
ECU E_5 has capabilities: {'HW_CANFD', 'HW_HSM', 'HW_FLEX'}
ECU E_6 has capabilities: {'HW_CANFD', 'HW_HSM', 'HW_FLEX'}
ECU E_7 has capabilities: {'HW_CANFD', 'HW_ETH', 'HW_HSM'}
ECU E_8 has capabilities: {'HW_CANFD', 'HW_HSM', 'HW_FLEX'}
ECU E_9 has capabilities: {'HW_ETH'}
ECU E_10 has capabilities: {'HW_ETH'}
Component C_2 requires features: set()
Component C_3 requires features: set()
Component C_4 requires features: {'HW_FLEX'}
Component C_5 requires features: {'HW_ETH'}
Component C_6 requires features: set()
Component C_7 requires features: set()
Component C_8 requires features: {'HW_ETH'}
Component C_9 requires features: {'HW_ETH'}
Component C_10 requires features: set()
Component C_11 requires features: {'HW_FLEX'}
Component C_1

### C5: ASIL Safety Constraint: 
Logic: A Component cannot be assigned to an ECU with a lower ASIL Capability.
If Required_ASIL > Available_ASIL, then x[i][j] must be 0.

In [9]:
for i in COMPONENTS:
    # Get the integer value of the component's requirement (e.g., 'D' -> 4)
    req_level = ASIL_MAP[REQ_ASIL[i]]
    
    for j in ECUS:
        # Get the integer value of the ECU's capability (e.g., 'QM' -> 0)
        cap_level = ASIL_MAP[ECU_ASIL_CAPABILITY[j]]
        
        # Check for violation
        if req_level > cap_level:
            # Forbidden Assignment: Force x to 0
            # This effectively removes this path from the solver's options.
            model += x[i][j] == 0, f"C6_ASIL_Mismatch_{i}_{j}"


### C6: ECU Utilization Link:
The binary variable $y_j$ must be correctly linked to $x_{ij}$ to track active ECUs for the cost objective (O3). $M$ is a sufficiently large number (e.g., the total number of components $I$).
$$\sum_{i=1}^{I} x_{ij} \le M \cdot y_{j} \quad \forall j \in \{1, \dots, J\}$$

In [10]:
for j in ECUS:
    # M * y[j] must be greater than or equal to the total number of components assigned to ECU j.
    model += lp.lpSum(x[i][j] for i in COMPONENTS) <= M * y[j], f"C5_ECU_Used_Link_{j}"

## Objective Functions
The problem is structured as a Multi-Objective Optimization. The final goal is to minimize a weighted sum of the following conflicting objectives:

### O1: Minimize Inter-ECU Communication Cost ($Z_1$): 
Minimize the total communication volume and associated latency penalties incurred when components $i$ and $k$ that communicate heavily are assigned to different ECUs ($j \neq l$).

$$\min Z_1 = \sum_{i=1}^{I} \sum_{k=1}^{I} \sum_{j=1}^{J} \sum_{l=1}^{J} (\text{CommVol}_{i,k} \cdot x_{ij} \cdot x_{kl} \cdot \delta_{j \neq l})$$

(Note: The term $x_{ij} \cdot x_{kl}$ is non-linear and requires linearization techniques or a non-linear solver.)

In [11]:
Z1_CommCost = lp.lpSum(COMM_VOLUME.get((i, k), 0) * w[(i, k, j, l)] 
                       for i, k in COMM_VOLUME.keys() for j in ECUS for l in ECUS if j != l)

### O2: Minimize Resource Imbalance (Load Balancing) ($Z_2$):

Minimize the variance of resource utilization across all active ECUs to ensure balanced workload distribution, enhancing system responsiveness and stability.

$$\min Z_2 = \sum_{j=1}^{J} \left( \text{Utilization}_j - \text{AvgUtilization} \right)^2$$

(Note: This is a non-linear (quadratic) objective, often approximated or converted to a Minimax linear problem in ILP, or handled via meta-heuristics.)

Its become this like after thought about every resource $d_{j,r}$ is deficit b/w avg utilization and real utilization and $r$ is the resource 

$$\min Z_2 = \sum_{r} \sum_{j} (w_{r} \cdot d_{j,r})$$

Another linearization is here: Approx quadratic function as a aboslute function $d = |A - B|$ but this is V shape function which is cannot be differentiable. So we need to add two different constraints

1) $d \ge A - B$ 
2) $d \ge B - A$  (or $d \ge -(A - B)$)


In [12]:
# --- STEP 1: Calculate System-Wide Target Utilization for All Resources (%) ---

total_cpu_cap = sum(MAX_CPU.values())
total_ram_cap = sum(MAX_RAM.values())
total_rom_cap = sum(MAX_ROM.values())
total_gpu_cap = sum(MAX_GPU.values())

TARGET_UTIL_CPU = sum(REQ_CPU.values()) / total_cpu_cap
TARGET_UTIL_RAM = sum(REQ_RAM.values()) / total_ram_cap
TARGET_UTIL_ROM = sum(REQ_ROM.values()) / total_rom_cap 
# Handle GPU separately as capacity might be 0
TARGET_UTIL_GPU = sum(REQ_GPU.values()) / total_gpu_cap if total_gpu_cap > 0 else 0

print(f"Target Utilizations -> CPU: {TARGET_UTIL_CPU:.1%}, RAM: {TARGET_UTIL_RAM:.1%}, ROM: {TARGET_UTIL_ROM:.1%}, GPU: {TARGET_UTIL_GPU:.1%}")

# --- STEP 2: Define Deviation Variables ---
# Continuous variables to track deviation from the target average for each resource.
d_cpu = lp.LpVariable.dicts("Dev_CPU", ECUS, lowBound=0, cat=lp.LpContinuous)
d_ram = lp.LpVariable.dicts("Dev_RAM", ECUS, lowBound=0, cat=lp.LpContinuous)
d_rom = lp.LpVariable.dicts("Dev_ROM", ECUS, lowBound=0, cat=lp.LpContinuous) # New: ROM Deviation
d_gpu = lp.LpVariable.dicts("Dev_GPU", ECUS, lowBound=0, cat=lp.LpContinuous)

# --- STEP 3: Add Linearization Constraints ---
# Ensure d_res[j] >= |(Used / Capacity) - Target_Util|

for j in ECUS:
    # --- CPU Constraints ---
    used_cpu = lp.lpSum(REQ_CPU[i] * x[i][j] for i in COMPONENTS)
    if MAX_CPU[j] > 0:
        util_cpu = used_cpu * (1.0 / MAX_CPU[j])
        model += d_cpu[j] >= util_cpu - TARGET_UTIL_CPU
        model += d_cpu[j] >= TARGET_UTIL_CPU - util_cpu
    else:
        model += d_cpu[j] == 0 

    # --- RAM Constraints ---
    used_ram = lp.lpSum(REQ_RAM[i] * x[i][j] for i in COMPONENTS)
    if MAX_RAM[j] > 0:
        util_ram = used_ram * (1.0 / MAX_RAM[j])
        model += d_ram[j] >= util_ram - TARGET_UTIL_RAM
        model += d_ram[j] >= TARGET_UTIL_RAM - util_ram
    else:
        model += d_ram[j] == 0

    # --- ROM Constraints (New) ---
    used_rom = lp.lpSum(REQ_ROM[i] * x[i][j] for i in COMPONENTS)
    if MAX_ROM[j] > 0:
        util_rom = used_rom * (1.0 / MAX_ROM[j])
        model += d_rom[j] >= util_rom - TARGET_UTIL_ROM
        model += d_rom[j] >= TARGET_UTIL_ROM - util_rom
    else:
        model += d_rom[j] == 0

    # --- GPU Constraints ---
    used_gpu = lp.lpSum(REQ_GPU[i] * x[i][j] for i in COMPONENTS)
    if MAX_GPU[j] > 0:
        util_gpu = used_gpu * (1.0 / MAX_GPU[j])
        model += d_gpu[j] >= util_gpu - TARGET_UTIL_GPU
        model += d_gpu[j] >= TARGET_UTIL_GPU - util_gpu
    else:
        model += d_gpu[j] == 0

# --- STEP 4: Z2 Objective (Weighted Sum) ---
# Weights determine the priority of balancing each resource.
# Pragmatically: RAM > CPU > GPU > ROM
W_CPU = 1.0
W_RAM = 1.5  # Memory bottlenecks are often fatal; prioritize balancing here.
W_GPU = 1.2  # AI workloads are heavy; keeping this balanced is good for thermal management.
W_ROM = 0.8  # Storage is usually static; less critical to balance perfectly than RAM.

Z2_LoadBalance = lp.lpSum(
    (W_CPU * d_cpu[j]) + 
    (W_RAM * d_ram[j]) + 
    (W_ROM * d_rom[j]) + # New: Include ROM in sum
    (W_GPU * d_gpu[j]) 
    for j in ECUS
)

Target Utilizations -> CPU: 41.8%, RAM: 96.2%, ROM: 5.6%, GPU: 68.5%


### O3: Minimize Total System Cost and Complexity Index ($Z_3$)

Minimize the total hardware acquisition cost, factoring in penalties for excessive wiring/complexity and residual safety risk associated with the chosen ECU count.
$$\min Z_3 = \sum_{j=1}^{J} (\text{Cost}_j \cdot y_j) $$

In [13]:
# The total cost includes the base cost of the ECU plus the complexity/risk penalty 
# (represented here as a fixed amount added to the base cost, multiplied by the usage variable y_j)
Z3_SystemCost = lp.lpSum(ECU_BASE_COST[j] * y[j] for j in ECUS)

***Overall Optimization Goal (Weighted Sum Method)***
The combined objective is to minimize the weighted sum $Z$:$$\min Z = \lambda_1 \cdot Z_1 + \lambda_2 \cdot Z_2 + \lambda_3 \cdot Z_3$$($\lambda_1, \lambda_2, \lambda_3$ are pre-defined positive weighting factors determined by architectural priorities.)

In [14]:
# Weights 
LAMBDA_1 = 1.0 #Communication Cost Weight
LAMBDA_2 = 1.0 #Load Balancing Weight
LAMBDA_3 = 1.0 #Complexity Penalty Weight

# Combine the objectives using the defined weights (LAMBDA_1 and LAMBDA_3)
model += LAMBDA_1 * Z1_CommCost + LAMBDA_2 * Z2_LoadBalance + LAMBDA_3 * Z3_SystemCost, "Total_Weighted_Cost"

In [15]:
# --- Step 4: Solve & Analyze (Corrected Logic) ---

# Solver Setup: 300 seconds limit
# msg=1 allows seeing the solver log (CBC log)
solver = lp.PULP_CBC_CMD(timeLimit=300, msg=1)

print("Solver starting... (Max 300s)")
model.solve(solver)

# --- Optimization Results ---
print("\n--- OPTIMIZATION RESULTS ---")

# Status Durumunu Kontrol Et (Sayısal Değer)
# 1: Optimal, 0: Not Solved, -1: Infeasible, -2: Unbounded, -3: Undefined
status_code = model.status
status_text = lp.LpStatus[status_code]

print(f"Solver Status Code: {status_code} ({status_text})")

# MANTIK DÜZELTME: Sadece 'Optimal' (veya süre yetmediyse 'Not Solved' ama feasible) ise yazdır.
# Infeasible (-1) ise kesinlikle yazdırma, çünkü atama yok.
if status_code == lp.LpStatusOptimal:
    print(f"SUCCESS: Optimal Solution Found! (Total Weighted Cost: {lp.value(model.objective):.2f})")
    
    # --- Atamaları Çek ve Yazdır ---
    assignment_results = {}
    for i in COMPONENTS:
        for j in ECUS:
            # PuLP'ta değer None ise 0 kabul et, float hatası için > 0.99 kontrolü yap
            val = x[i][j].varValue
            if val is not None and val > 0.99:
                assignment_results[i] = j

    from collections import defaultdict
    ecu_groups = defaultdict(list)

    for comp, ecu in assignment_results.items():
        ecu_groups[ecu].append(comp)

    # 1. Component Assignment Yazdır
    print("\nComponent Assignment (Grouped by ECU):")
    if not ecu_groups:
        print("  WARNING: Status is Optimal but no assignments found? Check constraints.")
        
    for ecu, components in ecu_groups.items():
        # components.sort(key=lambda x: int(x.split('_')[1])) # İsteğe bağlı sıralama
        print(f"{ecu}: {components}")
        
    # 2. Resource Load Yazdır
    print("\nECU Usage and Resource Load:")
    for j in ECUS:
        # ECU kullanılıyor mu?
        y_val = y[j].varValue
        if y_val is not None and y_val > 0.99:
            assigned_components = [i for i in COMPONENTS if assignment_results.get(i) == j]
            
            # Kaynak Toplamları
            total_cpu = sum(REQ_CPU[i] for i in assigned_components)
            total_ram = sum(REQ_RAM[i] for i in assigned_components)
            total_rom = sum(REQ_ROM[i] for i in assigned_components)
            total_gpu = sum(REQ_GPU[i] for i in assigned_components)
            
            print(f"  {j} (Used):")
            print(f"    CPU Load: {total_cpu:8.1f} / {MAX_CPU[j]:.1f} ({total_cpu/MAX_CPU[j]*100:5.1f}%)")
            print(f"    RAM Load: {total_ram:8.1f} / {MAX_RAM[j]:.1f} ({total_ram/MAX_RAM[j]*100:5.1f}%)")
            print(f"    ROM Load: {total_rom:8.1f} / {MAX_ROM[j]:.1f} ({total_rom/MAX_ROM[j]*100:5.1f}%)")
            if MAX_GPU[j] > 0:
                print(f"    GPU Load: {total_gpu:8.1f} / {MAX_GPU[j]:.1f} ({total_gpu/MAX_GPU[j]*100:5.1f}%)")

elif status_code == -1: # Infeasible
    print("\nFAILURE: Problem is INFEASIBLE.")
    print("Hard Constraint ihlali var. Kaynaklar (RAM/CPU) talebi karşılamıyor veya kurallar çakışıyor.")
    print("Çözüm kümesi boş olduğu için atama yapılamadı.")

elif status_code == 0: # Not Solved (Time Limit?)
    print("\nWARNING: Solver stopped (Time Limit or Interrupted).")
    if lp.value(model.objective) is not None:
         print("Partial solution might exist but isn't guaranteed optimal.")
    else:
         print("No feasible solution found yet.")
else:
    print(f"\nError: Solver returned status {status_text}")

Solver starting... (Max 300s)
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/frk/optimization/.optimization/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/7d2d911f2f4e47659bcaf9aab6dab289-pulp.mps -sec 300 -timeMode elapsed -branch -printingOptions all -solution /tmp/7d2d911f2f4e47659bcaf9aab6dab289-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 126198 COLUMNS
At line 551444 RHS
At line 677638 BOUNDS
At line 719989 ENDATA
Problem MODEL has 126193 rows, 42390 columns and 298555 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 300
Option for timeMode changed from cpu to elapsed
Continuous objective value is 125.564 - 0.09 seconds
Cgl0002I 134 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 4093 strengthened rows, 11880 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5024 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bo